<a href="https://colab.research.google.com/github/rodiwaa/learnings-pocs/blob/main/notebooks/adk_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# plan
# - create simple greetin_agent
# - create tool to print hello or hey
# - wrap in functool
# - attach to greeting_agent
# - print
# - stream

In [ ]:
! pip install google-adk google-generativeai

In [36]:
from google.adk.agents import Agent, ParallelAgent, LlmAgent, SequentialAgent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from IPython.display import display, Markdown

In [25]:
from getpass import getpass
# from google.generativeai import genai
import google.generativeai as genai
import os

api_key = getpass("enter google api_key: ")

genai.configure(api_key=api_key)

os.environ["GOOGLE_API_KEY"] = api_key

print("google api key set")

enter google api_key: ··········
google api key set


In [ ]:
# agent

# session

# userid

# runner

# run

In [40]:
# --- Agent Definition ---

# agents
def create_weather_agent():
    """Create the friendly news fetch agent"""
    return Agent(
        name="weather",
        model="gemini-2.5-flash",
        description="Agent specialized in fetching day forecast of given place.",
        instruction="""
        You are the "Weather Forecaster" Agent 🚗 - a specialized AI assistant that creates forecast weather.

        Your Mission:
        Get weather forecast for given location. Present in friendly manner.

        Guidelines:
        1. **General Weather**: Focus on general weather conditions like hot, sunny, rainy.
        2. **Temperature**: Create morning, afternoon, and evening forecast for day.
        3. **AQI Pollution Info**: Find relevant info on pollution levels. Mention AQI levels with categories (Good, Safe, Warning, Critical).
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN summary in MARKDOWN FORMAT with clear time blocks.
        """,
        tools=[google_search],
        output_key="weather-results"
    )

def create_news_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in finding news headlines for major categories.",
        instruction="""
        You are the "News" Generator 🚗 - a specialized AI assistant that brings important news for the day.

        Your Mission:
        Get top 2 headlines and new article URL for categories - World, Sports (Cricket), Business, Technology, Local (Mumbai) only.

        Guidelines:
        1. **Headlines and URL**: Make sure headlines summarise gist of news item. Provide clickable URL.
        2. **Categories**: Stick to given categories only - World, Sports (Cricket), Business, Technology, Local (Mumbai) only.
        3. **Avoid**: Avoid these topics - Politics, Bollywood.

        RETURN news in MARKDOWN FORMAT with clear healines and working URLs.
        """,
        tools=[google_search],
        output_key="news-results"
    )

weather_agent = create_weather_agent()
news_agent = create_news_agent()
print(f"🧞 Agent '{weather_agent.name}' is created and ready to bring you weather updates!")
print(f"🧞 Agent '{news_agent.name}' is created and ready to bring you news!")


def create_root_agent():
  """Root agent to run weather, news agents in parallel"""
  return ParallelAgent(
      name="orchestrator",
      # model="gemini-2.5-flash",
      description="Runs weather and news agents in parallel.",
      sub_agents=[
          weather_agent, news_agent
      ]
  )

orchestrator_agent = create_orchestrator_agent()
print(f"🧞 Agent '{orchestrator_agent.name}' is created and ready!")

merger_agent = LlmAgent(
     name="MergerAgent",
     model="gemini-2.5-flash",
     instruction="""You are an AI Assistant responsible for combining news, weather findings into a structured report.

 Your primary task is to synthesize the following results, clearly attributing findings to their source areas. Structure your response using headings for each topic. Ensure the report is coherent and integrates the key points smoothly.

 **Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the 'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these specific results.**

 **Input Summaries:**

 *   **News:**
     {news-results}

 *   **Weather:**
     {weather-results}

 **Output Format:**

 ## Summary of Recent Sustainable Technology Advancements

 ### News Results
 (Based on RenewableEnergyResearcher's findings)
 [Synthesize and elaborate *only* on the renewable energy input summary provided above.]

 ### Weather Results
 (Based on EVResearcher's findings)
 [Synthesize and elaborate *only* on the EV input summary provided above.]

 ### Overall Summary
 [Provide a brief (1-2 sentence) concluding statement that connects *only* the results presented above.]

 MARKDOWN output *only* the structured report following this format. Do not include introductory or concluding phrases outside this structure, and strictly adhere to using only the provided input summary content.
 """,
     description="Combines research findings from news, weather agents into a structured strictly grounded on provided inputs.",
     # No tools needed for merging
     # No output_key needed here, as its direct response is the final output of the sequence
 )


🧞 Agent 'weather' is created and ready to bring you weather updates!
🧞 Agent 'day_trip_agent' is created and ready to bring you news!
🧞 Agent 'orchestrator' is created and ready!


In [ ]:
# run both

In [27]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )
    final_response = ""
    try:
       async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):

         if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
      final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

session_service = InMemorySessionService()
my_user_id = "rodi_adventurer_001"

In [ ]:
### NOT RUNNING THESE AGENTS INDIVIDUALLY NOW. RUN ORCHESTRATOR AGENT INSTEAD ###
async def run_weather_agent():
    # Create a new, single-use session for this query
    weather_session = await session_service.create_session(
        app_name=weather_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me next 3 days weather forecast for Mumbai"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_agent()

In [ ]:
### NOT RUNNING THESE AGENTS INDIVIDUALLY NOW. RUN ORCHESTRATOR AGENT INSTEAD ###

async def run_news_agent():
    # Create a new, single-use session for this query
    news_session = await session_service.create_session(
        app_name=news_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, news_session, my_user_id)

await run_news_agent()

In [ ]:
### NOT RUNNING THIS ONE EITHER. SEE SEQUENTIAL AGENT BELOW ###
async def run_orchestrator_agent():
    # Create a new, single-use session for this query
    orchestrator_session = await session_service.create_session(
        app_name=orchestrator_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines and weather updates for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, orchestrator_session, my_user_id)

await run_orchestrator_agent()

In [42]:
 # --- 4. Create the SequentialAgent (Orchestrates the overall flow) ---
 # This is the main agent that will be run. It first executes the ParallelAgent
 # to populate the state, and then executes the MergerAgent to produce the final output.
root_agent = SequentialAgent(
     name="NewsAndWeatherPipeline",
     # Run parallel research first, then merge
     sub_agents=[orchestrator_agent, merger_agent],
     description="Coordinates parallel news and weather the results."
 )

root_agent = root_agent

ValidationError: 1 validation error for SequentialAgent
  Value error, Agent `orchestrator` already has a parent agent, current parent: `ResearchAndSynthesisPipeline`, trying to add: `NewsAndWeatherPipeline` [type=value_error, input_value={'name': 'NewsAndWeatherP...d weather the results.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [43]:
# Running root/ sequential agent

async def run_root_agent():
    # Create a new, single-use session for this query
    root_session = await session_service.create_session(
        app_name=root_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines and weather updates for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, root_session, my_user_id)

await run_orchestrator_agent()

NameError: name 'run_orchestrator_agent' is not defined